In [ ]:
'''
2. Neural Network Classifier with Keras
Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using Keras. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.
'''

In [5]:
# Training a multiclass Classifier
# Using Keras to construct a feedforward neural network with an output layer with soft-max activation functions

# Load libraries
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers

#import json library to read data in jsonl file
import json
#import pandas library
import pandas as pd

In [6]:
#read in the data as a dataframe
filename = "/home/arindam/Documents/mygithub/bu_dsc/data/raw/categorized-comments.jsonl"
with open(filename, 'r') as f:
    jsonl_list = list(f)

list1 = []
for obj in jsonl_list:
    res = json.loads(obj)
    list1.append(res)
    
comments = pd.DataFrame(list1)

#display the first few rows of data
comments.head()
# len(list1)

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [7]:
#Convert text to lowercase and romove punctuation
#define a function to clean the text
# import the required libraries here
#import regular expressions library
import re

def clean_text(text):
    """
    Remove punctuations and special characters, makes lower case
    Args: text
    Output: text
    """
    text=text.lower() #makes text lowercase
    text=re.sub('\\d|\\W+|_',' ',text) #removes extra white space
    text=re.sub('[^a-zA-Z]'," ", text) #removes any non-alphabetic characters
    return text

In [44]:
# Cleaning texts in the comments
# Using the transformed column for the model
# Testing the functions

sample_size = 50000
sample_cmnts = comments[:sample_size]
sample_cmnts['cleaned']=sample_cmnts['txt'].apply(clean_text)
# creating a dictionary to replace the string values to numeric
d = {'sports':1,'science_and_technology':2,'video_games':3}
sample_cmnts['ncat'] = sample_cmnts['cat'].map(d)
sample_cmnts.head

<bound method NDFrame.head of           cat                                                txt  \
0      sports  Barely better than Gabbert? He was significant...   
1      sports  Fuck the ducks and the Angels! But welcome to ...   
2      sports  Should have drafted more WRs.\n\n- Matt Millen...   
3      sports            [Done](https://i.imgur.com/2YZ90pm.jpg)   
4      sports                                      No!! NOO!!!!!   
...       ...                                                ...   
49995  sports  Florida State\nAlabama\nMichigan\nAlabama\nAla...   
49996  sports                                        Never does.   
49997  sports  I think Tosh is our best one left. Napier is g...   
49998  sports      Close, and we could really use a tight end...   
49999  sports  Yeah but that didn't really end all that well ...   

                                                 cleaned  ncat  
0      barely better than gabbert he was significantl...     1  
1      fuck the ducks a

In [71]:
sample_cmnts.ncat.value_counts()

2    25111
1    24889
Name: ncat, dtype: int64

In [70]:
output_activation=sample_cmnts.ncat.unique()[1]
output_activation

2

In [50]:
# Get the target name
from sklearn.model_selection import train_test_split

# Creating the features from the data set
features, target = sample_cmnts.cleaned, sample_cmnts.ncat

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

number_of_features = 2000

tfidf = TfidfVectorizer(max_features = number_of_features, stop_words=stopwords.words('english'))

features = tfidf.fit_transform(features).toarray()
features.shape

(50000, 2000)

In [52]:
# Make test and training split (30:70)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(features,target, random_state=0, test_size = 0.3)

In [72]:
# X_train.shape
# X_test.shape
# y_train.shape
# y_test.shape

(35000,)

In [53]:
# Start Neural Network
network = models.Sequential()

In [73]:
# Adding fully connected input layer with a ReLU activation function
network.add(layers.Dense(units=100,
                         activation='relu',
                         input_shape=(number_of_features,)))

In [77]:
# Add fully connected layer with ReLU activation function
network.add(layers.Dense(units=200, activation="relu"))

In [78]:
# Add fully connected layer with a softmax activation function
network.add(layers.Dense(units=output_activation, activation="softmax"))

In [79]:
# Compile neural network
network.compile(loss="categorical_crossentropy", # Cross-entropy
                optimizer="rmsprop", # Root Mean Square Propagation
                metrics=["accuracy"]) # Accuracy performance metric

In [80]:
# Train neural network
history = network.fit(X_train, # Features
                      y_train, # Target
                      epochs=3, # Three epochs
                      verbose=1, # show output
                      batch_size=100, # Number of observations per batch
                      validation_data=(X_test, y_test)) # Test data

Epoch 1/3


ValueError: in user code:

    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
        loss = self.compiled_loss(
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/compile_utils.py:205 __call__
        loss_value = loss_obj(y_t,

In [62]:
# Saving the Model

model_path="/home/arindam/Documents/mygithub/bu_dsc/models"
model_name="NN_keras.h5"
filename = model_path + "/" + model_name 
# print(filename)
network.save(filename)

In [63]:
# Load a saved model
from keras.models import load_model
NN_clf = load_model(filename)


In [64]:
# Predicting the test set for the classifier
y_pred = NN_clf.predict(X_test)
# y_pred

In [65]:
# Displaying the result metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

print("Confusion Matrix")
print("================")
print(confusion_matrix(y_test,y_pred))
print("Classification Report")
print("=====================================================")

print(classification_report(y_test,y_pred))
print("Accuracy Score")
print("=====")

print(accuracy_score(y_test, y_pred))

Confusion Matrix
[[7441    0]
 [7559    0]]
Classification Report
              precision    recall  f1-score   support

           1       0.50      1.00      0.66      7441
           2       0.00      0.00      0.00      7559

    accuracy                           0.50     15000
   macro avg       0.25      0.50      0.33     15000
weighted avg       0.25      0.50      0.33     15000

Accuracy Score
=====
0.49606666666666666
